# Seleccion compuestos multitarget
Después de obtener la predicción usando cada una de las proteínas, es importante unificar los datos en un archivo .CSV según la cantidad de proteínas con las que desean realizar el análisis, es importante generar columnas que especifiquen que proteína es en el archivo.CSV como se muestra en la siguiente imagen.


como se obserba en la imagen anterior, se unifica la informacion de cada proteina en Bioactividad, Probabilidad Bioactividad, despues de unificar todos estos datos en un solo archivo.CSV, genere una columna en este archivo llamada Bioactividad_en3_proteinas, y en esta usando una funcion condicional coloque multitarget, donde las suma de las columnas bioactividad sea mayor o igual a 3, haciendo referencia que son bioactivas en tres proteinas, este archivo puede llamarlo prediccion_enamine.

## Requisitos previos:
Asegúrate de tener un entorno de Python con las siguientes bibliotecas instaladas: matplotlib, pandas, seaborn, scikit-learn, imbalanced-learn, optuna.

In [ ]:
import pickle
import pickletools
import os
import pandas as pd
import matplotlib.pyplot as plt

## Clasificacion compuestos multitarget
El código comienza importando las bibliotecas necesarias, como pandas y matplotlib.pyplot. A continuación, carga un archivo CSV llamado 'prediccion_enamine.csv' en un DataFrame llamado df, filtrando las filas donde la columna 'Bioactividad_en3_proteinas' es igual a 'multitarget'. Se definen las columnas de probabilidad de bioactividad para diferentes proteínas y se calcula la probabilidad multitarget para cada fila del DataFrame. Las combinaciones de bioactividades válidas se calculan y almacenan en un DataFrame junto con sus frecuencias de aparición. Los resultados se guardan en archivos CSV, como 'combinaciones.csv' y 'probabilidadcatalogo.csv'. Se realizan análisis adicionales de combinaciones específicas y se generan gráficos de histogramas para estas combinaciones, marcando las probabilidades superiores a 0.7 en rojo. Los resultados finales se unifican en un solo DataFrame y se guardan en un archivo CSV llamado 'catalogomultitarget.csv'. Finalmente, se visualizan los resultados unificados en la consola. El código se ejecuta colocando el archivo CSV 'prediccion_enamine.csv' en la misma carpeta y luego ejecutándolo para realizar el análisis y generar los resultados.

In [ ]:
os.chdir('/home/aamaya/RN')

df = pd.read_csv('prediccion_enamineSINREGLAS.csv')

os.chdir('/home/aamaya/RN/combinacionesmayores0.7')

multitarget_df = df[df['Bioactividad_en3_proteinas'].str.lower() == 'multitarget']


columnas_probabilidad = ['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_BACE1', 'Probabilidad_Bioactividad_GSK3B', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_MAO', 'Probabilidad_Bioactividad_RN']


def calcular_probabilidad_multitarget(row):

    probabilidades_validas = []
    

    for col in columnas_probabilidad:
        valor = row[col]
        if valor >= 0.5:
            probabilidades_validas.append(valor)
    

    if len(probabilidades_validas) > 0:
        probabilidad = sum(probabilidades_validas) / len(probabilidades_validas)
    else:
        probabilidad = 0.0
    
    return probabilidad


multitarget_df['Probabilidad_Multitarget'] = multitarget_df.apply(calcular_probabilidad_multitarget, axis=1)


combinaciones = []

for index, row in multitarget_df.iterrows():
    combinacion = []
    for col in columnas_probabilidad:
        if row[col] >= 0.7:  
            combinacion.append(col)
    if combinacion:
        combinaciones.append(', '.join(combinacion))


combinaciones_df = pd.DataFrame({'Combinacion': combinaciones})


combinaciones_count = combinaciones_df['Combinacion'].value_counts().reset_index()
combinaciones_count.columns = ['Combinacion', 'Count']


total_multitarget_rows = len(multitarget_df)
combinaciones_count['Porcentaje'] = (combinaciones_count['Count'] / total_multitarget_rows) * 100


combinaciones_count.to_csv('combinaciones.csv', index=False)


multitarget_df[['Catalog_ID', 'Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_BACE1', 'Probabilidad_Bioactividad_GSK3B', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_MAO', 'Probabilidad_Bioactividad_RN','Probabilidad_Multitarget']].to_csv('probabilidadcatalogo.csv', index=False)


print("Combinaciones de Bioactividades Distintas:")
print(combinaciones_count)


print("\nProbabilidad Multitarget:")
print(multitarget_df[['Catalog_ID', 'Probabilidad_Multitarget']].head(15))


combinaciones_especificas = [['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_RN'],
                             ['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_MAO'],
                             ['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_BACE1', 'Probabilidad_Bioactividad_CB2']]

nombres_combinaciones_especificas = ['Probabilidad_Bioactividad_ACHE_CB2_RN',
                                     'Probabilidad_Bioactividad_ACHE_CB2_MAO',
                                     'Probabilidad_Bioactividad_ACHE_BACE1_CB2']

for i, combinacion in enumerate(combinaciones_especificas):

    filtered_df = multitarget_df[
        (multitarget_df[combinacion[0]] >= 0.7) &
        (multitarget_df[combinacion[1]] >= 0.7) &
        (multitarget_df[combinacion[2]] >= 0.7) &
        (multitarget_df['Probabilidad_Multitarget'] > 0.7)
    ]
       

    total_datos = len(filtered_df)
    plt.title(f'Histograma de {nombres_combinaciones_especificas[i]} (Total de Datos: {total_datos})')
    

    filename = f'{nombres_combinaciones_especificas[i]}.csv'
    filtered_df[['Catalog_ID', 'Probabilidad_Multitarget']].to_csv(filename, index=False)
    print(f"Archivo '{filename}' creado con exito.")
    



probabilidad_df = pd.read_csv('probabilidadcatalogo.csv')


combinaciones_especificas = [['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_RN'],
                             ['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_CB2', 'Probabilidad_Bioactividad_MAO'],
                             ['Probabilidad_Bioactividad_ACHE', 'Probabilidad_Bioactividad_BACE1', 'Probabilidad_Bioactividad_CB2']]

nombres_combinaciones_especificas = ['ACHE,CB2,RN-a7',
                                     'ACHE,CB2,MAO-A',
                                     'ACHE,BACE1,CB2']


for i, combinacion in enumerate(combinaciones_especificas):

    filtered_df = probabilidad_df[
        (probabilidad_df[combinacion[0]] >= 0.7) &
        (probabilidad_df[combinacion[1]] >= 0.7) &
        (probabilidad_df[combinacion[2]] >= 0.7) 
    ]
    

    probabilidad_multitarget = filtered_df['Probabilidad_Multitarget']
        

    total_data = len(probabilidad_multitarget)
    

    total_data_above_0_7 = len(probabilidad_multitarget[probabilidad_multitarget >= 0.7])
    total_data_above_0_6 = len(probabilidad_multitarget[probabilidad_multitarget < 0.7])
    

    plt.hist(probabilidad_multitarget[probabilidad_multitarget >= 0.7], bins=20, range=(0.5, 1), color='red', alpha=0.7, label=f'>=0.7 (Total: {total_data_above_0_7})')
    plt.hist(probabilidad_multitarget[probabilidad_multitarget < 0.7], bins=20, range=(0.5, 1), color='green', alpha=0.7, label=f'<0.7 (Total: {total_data_above_0_6})')
    

    plt.xlabel('Average')
    plt.ylabel('Quantity')
    plt.title(f'Histogram of Average for {nombres_combinaciones_especificas[i]} (Total Data: {total_data})')
    plt.legend()
    

    plt.savefig(f'{nombres_combinaciones_especificas[i]}_histogram.png')  
    plt.close()  

    print(f"Histogram for '{nombres_combinaciones_especificas[i]}' created successfully.")
    

combinacion1_df = pd.read_csv('Probabilidad_Bioactividad_ACHE_CB2_RN.csv')
combinacion2_df = pd.read_csv('Probabilidad_Bioactividad_ACHE_CB2_MAO.csv')
combinacion3_df = pd.read_csv('Probabilidad_Bioactividad_ACHE_BACE1_CB2.csv')


catalogo_multitarget_df = pd.concat([combinacion1_df, combinacion2_df, combinacion3_df])


catalogo_multitarget_df.to_csv('catalogomultitarget.csv', index=False)


print("Tabla unificada de todas las combinaciones:")
print(catalogo_multitarget_df)

Script elaborado por Nelson Alejandro Amaya Orozco, como trabajo de grado, para el grupo de investigacion RamirezLAB